In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import h2o
import mpld3
mpld3.enable_notebook()
import matplotlib.pyplot as plt
from src import measure_repository
from src import repository_adapter
from matplotlib.dates import  DateFormatter
from matplotlib.dates import YearLocator
from scipy.ndimage import gaussian_filter
import numpy as np
import h2o
from h2o.estimators import H2OXGBoostEstimator
from h2o.grid.grid_search import H2OGridSearch


h2o.init(ip="local.h2o.air_data", port=54321)

Checking whether there is an H2O instance running at http://local.h2o.air_data:54321. connected.


H2O cluster uptime:,2 days 0 hours 26 mins
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.1
H2O cluster version age:,2 months and 17 days
H2O cluster name:,root
H2O cluster total nodes:,1
H2O cluster free memory:,10.42 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [3]:
import matplotlib.pyplot as plt
import numpy as np
from src import repository_adapter
from src import time_series_cross_validator
from decimal import Decimal


def validate_grid(metering_point_id):
    
    initial_dataset = repository_adapter.run_sql(
        """ SELECT
                result as value,            
                HOUR(date) as hour,
                MONTH(date) as month,
                WEEKDAY(date)+1 as weekday,
                ABS(WEEK(date) - 27) as week
                #WEEK(date) as week
            FROM air.air_quality_measures
            WHERE 
                metering_point_id = %s
                AND result IS NOT NULL
                AND YEAR(date) IN (2016)
            ORDER BY date ASC
            ;""" % (metering_point_id),
        ["value", "hour", "month", "weekday", "week"]
    )   

    x = initial_dataset.iloc[:, 1:]
    y = initial_dataset.iloc[:, [0]]

    time_series_cross_validator.validate(
        x,
        y,
        5,
        lambda x, y, train_dataset, validation_dataset: grid_callback(x, y, train_dataset, validation_dataset)
    )
    

def grid_callback(x, y, training_dataset, validation_dataset):
    
    print('Inputs:')
    print(training_dataset.columns.values)

    training_frame = h2o.H2OFrame(training_dataset.values)
    training_frame.set_names(training_dataset.columns.values.tolist())
    validation_frame = h2o.H2OFrame(validation_dataset.values)
    validation_frame.set_names(validation_dataset.columns.values.tolist())

    columns = training_frame.columns
    columns.pop(0)
    X = columns
    Y = 'value'

    grid = H2OGridSearch(
        H2OXGBoostEstimator(),
        {
            'ntrees': [100,200,300],
            'max_depth': [10,20,40],
            'learn_rate': [0.01,0.2,0.5],
            'sample_rate': [0.1, 0.5,  0.9],
            'col_sample_rate': [0.1, 0.5,  0.9],
            'col_sample_rate_per_tree': [0.5,  0.9],
            'stopping_rounds': [10,50,100],
            'min_rows': [10, 50],
            'stopping_tolerance': [0.1, 0.5,  0.9],
        },
        grid_id='depth_grid',
        # search_criteria={'strategy': "Cartesian"}
        search_criteria={
            'strategy': "RandomDiscrete",
            'max_models': 100,
            'max_runtime_secs': 500,
            'stopping_tolerance': 0.001,
            'stopping_rounds': 12,
#             'seed': 42
        }
    )
    grid.train(x=X, y=Y, training_frame=training_frame, validation_frame=validation_frame)
    calculated_grid = grid
    
#     [mae, residual_deviance, r2, mean_residual_deviance, rmsle, rmse, mse]
    calculated_grid = calculated_grid.get_grid(sort_by='mae', decreasing=True)
    print(calculated_grid.sorted_metric_table())
    
#             'ntrees': [100,300],
#             'max_depth': [5,10],
#             'learn_rate': [0.5,  0.9],
#             'sample_rate': [0.5,  0.9],
#             'col_sample_rate': [0.5,  0.9],
#             'col_sample_rate_per_tree': [0.5,  0.9],
#             'stopping_rounds': [10,100],
#             'min_rows': [10, 50],
#             'stopping_tolerance': [0.5,  0.9],
                
validate_grid(2)

Inputs:
['value' 'hour' 'month' 'weekday' 'week']
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
xgboost Grid Build progress: |████████████████████████████████████████████| 100%
Errors/Warnings building gridsearch model

Hyper-parameter: col_sample_rate, 0.5
Hyper-parameter: col_sample_rate_per_tree, 0.5
Hyper-parameter: learn_rate, 0.2
Hyper-parameter: max_depth, 10
Hyper-parameter: min_rows, 10.0
Hyper-parameter: ntrees, 300
Hyper-parameter: sample_rate, 0.9
Hyper-parameter: stopping_rounds, 50
Hyper-parameter: stopping_tolerance, 0.9
failure_details: None
failure_stack_traces: water.Job$JobCancelledException
	at hex.tree.xgboost.XGBoost$XGBoostDriver.scoreAndBuildTrees(XGBoost.java:407)
	at hex.tree.xgboost.XGBoost$XGBoostDriver.buildModelImpl(XGBoost.java:311)
	at hex.tree.xgboost.XGBoost$XGBoostDriver.buildModel(XGBoost.java:261)
	at hex.tree.xgboost.XGBoost$XGBoostDr

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'